In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [28]:
from selenium import webdriver
from PIL import Image
import os
import selenium
import time
import requests
import io
import hashlib

In [7]:
DRIVER_PATH = 'C:\\Tools\\chromedriver.exe' 

In [30]:
SEARCH_TERM = 'teddy bear'

In [9]:
DESTINATION = './images/bears'

In [10]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [11]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [12]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [13]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [31]:
search_and_download(search_term=SEARCH_TERM,driver_path=DRIVER_PATH,target_path=DESTINATION,number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://rukminim1.flixcart.com/image/800/800/ju04akw0/stuffed-toy/v/a/c/3-feet-stuff-teddy-bear-beautiful-giant-teddy-gift-for-original-imaff66axpq9jzqg.jpeg?q=90 - as ./images/bears\teddy_bear\06e8f8c919.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRMon-5pQIodyyFwMlZr4VI2SdAY--CGDBWyQ&usqp=CAU - as ./images/bears\teddy_bear\082a897160.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTnPeOfKPyELlLI6sGmsgz6zlY5EFFQEDvRXw&usqp=CAU - as ./images/bears\teddy_bear\4bab02a290.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTObNrieSbGeuM3RkFUY0mQavASJhrcb3ZMMQ&usqp=CAU - as ./images/bears\teddy_bear\1cc65562f6.jpg
SUCCESS - saved https://cdn.webshopapp.com/shops/289394/files/335674253/1600x2048x2/maileg-teddy-bear-dad.jpg - as ./images/bears\teddy_bear\8730ac3641.jpg
SUCCESS - saved https://encrypted-tbn0.gst

SUCCESS - saved https://cdn.webshopapp.com/shops/241159/files/293550546/steiff-amadeus-teddy-bear-blond.jpg - as ./images/bears\teddy_bear\c7d1b10df3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTF0k12Uze-vytWDxNhkCO4HTzMlgOpisJEWg&usqp=CAU - as ./images/bears\teddy_bear\c4506877ae.jpg
SUCCESS - saved https://archive-store.nl/pub/media/catalog/product/cache/207e23213cf636ccdef205098cf3c8a3/s/t/steiff-james-teddybear-archive-store-01.jpg - as ./images/bears\teddy_bear\4850a7b987.jpg
SUCCESS - saved https://i.pinimg.com/originals/39/01/e0/3901e062c1d63fa19e019f63a46c3a78.jpg - as ./images/bears\teddy_bear\72858bc639.jpg
SUCCESS - saved https://cdn.webshopapp.com/shops/260902/files/315913628/890x820x2/hermann-teddy-cuddly-animal-teddy-bear-with-sound.jpg - as ./images/bears\teddy_bear\4881af3133.jpg
SUCCESS - saved https://www.funbakings.nl/data/upload/Shop/images/jem-pop-it-teddy-bear--teddybeer.jpg - as ./images/bears\teddy_bear\2ae5ebe23e.jpg
SUCCESS - sa